Source : https://databank.worldbank.org/source/world-development-indicators
Prédiction de la population

In [ ]:
import pandas as pd
from IPython.display import display
# InteractiveShell.ast_node_interactivity = "all"


data = pd.read_csv("pop_1960_2023.csv")


data = data.drop(["Series Code"], axis=1)

# utiliser missMDA pour gérer les Nan

# Utiliser pandas.melt  pour remettre
# pandas.lreshape(data, groups, dropna=True)
# pandas.pivot(data, *, columns, index=<no_default>, values=<no_default>)
temp_dict = {}
# Renommer colonnes : 1960 [YR1960] => YR1960
for col in data.columns:
    if col[0].isnumeric():
        temp_dict = {col:col[0:4]}
        data = data.rename(columns=temp_dict)
        
# remettre données année en ligne et variable en colonne



data = pd.melt(data, id_vars=list(data.columns[0:3]), value_vars = list(data.columns[3:]))
data = data.rename(columns={"variable":"Year"})
print(data.columns)

# Pivot pour que les valeurs de "Series Name" deviennent des colonnes
data_pivot = data.pivot_table(
    index=["Country Name", "Country Code", "Year"],
    columns="Series Name",
    values="value",
    aggfunc='first'  
)

# a vérifier la logique 

data_pivot = data_pivot.reset_index()
display(data_pivot.head(10))
print(len(data))
#153792 enregistrements

len(data["Country Name"])


Index(['Country Name', 'Country Code', 'Series Name', 'Year', 'value'], dtype='object')


Series Name,Country Name,Country Code,Year,Access to electricity (% of population),Adjusted net national income (current US$),"Birth rate, crude (per 1,000 people)","Death rate, crude (per 1,000 people)",Electric power consumption (kWh per capita),Exports of goods and services (% of GDP),GDP per capita (current US$),"Life expectancy at birth, total (years)",Population growth (annual %),"Population, total","Suicide mortality rate (per 100,000 population)"
0,Afghanistan,AFG,1960,..,..,50.34,31.921,..,..,..,32.535,..,9035043,..
1,Afghanistan,AFG,1961,..,..,50.443,31.349,..,..,..,33.068,1.9622391202036,9214083,..
2,Afghanistan,AFG,1962,..,..,50.57,30.845,..,..,..,33.547,2.0445228359303,9404406,..
3,Afghanistan,AFG,1963,..,..,50.703,30.359,..,..,..,34.016,2.1052082248002,9604487,..
4,Afghanistan,AFG,1964,..,..,50.831,29.867,..,..,..,34.494,2.16119547210755,9814318,..
5,Afghanistan,AFG,1965,..,..,50.872,29.389,..,..,..,34.953,2.23370858013984,10036008,..
6,Afghanistan,AFG,1966,..,..,50.986,28.872,..,..,..,35.453,2.26965142881696,10266395,..
7,Afghanistan,AFG,1967,..,..,51.081,28.396,..,..,..,35.924,2.30666800699098,10505959,..
8,Afghanistan,AFG,1968,..,..,51.148,27.895,..,..,..,36.418,2.36068343389987,10756922,..
9,Afghanistan,AFG,1969,..,..,51.195,27.398,..,..,..,36.91,2.39272037816067,11017409,..


153792


#Gérer les NaN avec missMDA ...


'1'